## Patient Information Objects

### Patient (decrypted)

In [1]:
from models.patient import Patient

In [2]:
p1 = Patient('2712205f-755e-4897-acb3-926895b7d635')

In [3]:
p1.FIRST

'Emory494'

In [4]:
p1.get_age()

datetime.timedelta(days=29952, seconds=8212, microseconds=466776)

### Allergy (decrypted)

In [5]:
from models.allergy import Allergies

In [6]:
# A patient with multiple allergies
a1 = Allergies('df6b563d-1ff4-4833-9af8-84431e641e9c')

In [7]:
a1.allergies

In [8]:
# A patient with no allergies
a2 = Allergies('f0f3bc8d-ef38-49ce-a2bd-dfdda982b271')

In [9]:
a2.allergies

[]

by patient class

In [10]:
p1 = Patient('df6b563d-1ff4-4833-9af8-84431e641e9c')
p1.ALLERGY.get_n_allergies()

4

In [11]:
p2 = Patient('f0f3bc8d-ef38-49ce-a2bd-dfdda982b271')
p2.ALLERGY.get_n_allergies()

0

## Read from sql db

In [1]:
from app import connection
from sqlalchemy import text
import pandas as pd

In [34]:
with connection.OracleConnection() as engine:
    db_connection = engine.connect()
    try:
        statement = text("""SELECT * FROM BM3336.test1""")
        rs = db_connection.execute(statement).fetchall()
    except Exception as e:
        print("Error: " + str(e))
    finally:
        # close the db connection
        db_connection.close()
rs

[('aaa', 'bbb', '111'),
 ('aaa', 'bbb', '222'),
 ('aaa', 'aaa', '1111'),
 ('aaa', 'aaa', '11111')]

In [4]:
with connection.OracleConnection() as engine:
    db_connection = engine.connect()
    try:
        # execute SQL query in try statement 
        statement = text("""SELECT * FROM mitre_patient limit FETCH FIRST 2 ROWS ONLY""")
        df = pd.read_sql(statement,con=db_connection)
    except Exception as e:
        print("Error: " + str(e))
    finally:
        # close the db connection
        db_connection.close()
df

,patient_id,birthdate,deathdate,ssn,drivers,passport,prefix,first_name,last_name,suffix,...,birth_place,address,city,state,county,zip,lat,lon,healthcare_expenses,healthcare_coverage
0,b1bc3c89-4e7b-41fd-a58b-9bf4ce2b40a9,1973-07-04,None,999-85-9257,S99974338,X55702266X,Mr.,Louie190,Jacobi462,None,...,Lynn Massachusetts US,255 Ondricka Common,Deerfield,Massachusetts,Franklin County,None,42.574668,-72.562583,1018792.98,10929.18
1,50e7ede4-7ca1-4b2c-ae64-6b88b58eeaac,1965-10-15,None,999-85-8713,S99926370,X24151910X,Ms.,Marvel321,Wyman904,None,...,Wilbraham Massachusetts US,913 Schiller Well Apt 80,Burlington,Massachusetts,Middlesex County,01803,42.484740,-71.234137,1444325.28,4353.00


## Text generation

### read template 

In [5]:
import simplejson as json
from sqlalchemy import text

In [6]:
with open("template_section.json") as f:
    template = json.load(f)

In [7]:
dict_conditions = {
    "EQ" : "=",
    "GT" : ">",
    "LT" : "<"
}
dict_params = {
    "count" : "count(*)" 
    }

In [8]:
template

{'sections': {'DOCUMENT_HEADER': {'frequency': 'always'},
  'PATIENT': {'frequency': 'always',
   'table': 'MITRE_PATIENT',
   'sections': {'PATIENT_DEMOGRAPHICS': {'frequency': 'always',
     'title': 'Patient:\n'},
    'FEMALE_PATIENT': {'frequency': 'always',
     'when': ['{:GENDER:}', 'EQ', 'F']},
    'ALLERGY': {'frequency': 'always',
     'title': 'ALLERGY_TITLE',
     'table': 'MITRE_PATIENT_ALLERGY',
     'join': ['PATIENT_ID'],
     'sections': {'ALLERGY_0': {'when': ['count', 'EQ', 0]},
      'ALLERGY_1': {'when': ['count', 'EQ', 1]},
      'ALLERGY_N': {'when': ['count', 'GT', 1]}}}}}}}

In [9]:
for k,v in template['sections']['PATIENT'].items():
    print(k,v)

frequency always
table MITRE_PATIENT
sections {'PATIENT_DEMOGRAPHICS': {'frequency': 'always', 'title': 'Patient:\n'}, 'FEMALE_PATIENT': {'frequency': 'always', 'when': ['{:GENDER:}', 'EQ', 'F']}, 'ALLERGY': {'frequency': 'always', 'title': 'ALLERGY_TITLE', 'table': 'MITRE_PATIENT_ALLERGY', 'join': ['PATIENT_ID'], 'sections': {'ALLERGY_0': {'when': ['count', 'EQ', 0]}, 'ALLERGY_1': {'when': ['count', 'EQ', 1]}, 'ALLERGY_N': {'when': ['count', 'GT', 1]}}}}


In [10]:
p_section = template['sections']['PATIENT']
table_name = template['sections']['PATIENT']['table']
table_name

'MITRE_PATIENT'

In [11]:
p_section['sections']

{'PATIENT_DEMOGRAPHICS': {'frequency': 'always', 'title': 'Patient:\n'},
 'FEMALE_PATIENT': {'frequency': 'always', 'when': ['{:GENDER:}', 'EQ', 'F']},
 'ALLERGY': {'frequency': 'always',
  'title': 'ALLERGY_TITLE',
  'table': 'MITRE_PATIENT_ALLERGY',
  'join': ['PATIENT_ID'],
  'sections': {'ALLERGY_0': {'when': ['count', 'EQ', 0]},
   'ALLERGY_1': {'when': ['count', 'EQ', 1]},
   'ALLERGY_N': {'when': ['count', 'GT', 1]}}}}

In [12]:
with open("template_patient_info.json") as f:
    template = json.load(f)

In [13]:
for k,v in template['PATIENT_DEMOGRAPHICS'].items():
    print(k)

part1
part2
part3


In [14]:
string = template['PATIENT_DEMOGRAPHICS']['part1']
string[0]

'The patient, {:FIRST_NAME:}, is a {:BIRTHDATE:} year old {:Gender:} who presents with {:HEALTHCARE_COVERAGE:}.'

In [15]:
import re

def find_words(text):
    pattern = r'{:(.*?):}'
    matches = re.finditer(pattern, text)
    words = []
    for match in matches:
        word = match.group(1)
        position = match.start()
        word_len = len(word)+4
        words.append((position,word,word_len))
    return words

key_words_info = find_words(string[0])
key_words_info


[(13, 'FIRST_NAME', 14),
 (34, 'BIRTHDATE', 13),
 (57, 'Gender', 10),
 (86, 'HEALTHCARE_COVERAGE', 23)]

In [16]:
cols = []
for i in key_words_info:
    cols.append(i[1])
cols

['FIRST_NAME', 'BIRTHDATE', 'Gender', 'HEALTHCARE_COVERAGE']

In [17]:
def get_columns_str(list:cols):
    if len(cols) > 1:
        return ','.join(cols)
    else:
        return cols[0]
col_str = get_columns_str(cols)
col_str

'FIRST_NAME,BIRTHDATE,Gender,HEALTHCARE_COVERAGE'

In [18]:
table_name

'MITRE_PATIENT'

In [21]:
with connection.OracleConnection() as engine:
    db_connection = engine.connect()
    try:
        sql = f"""SELECT {col_str} FROM {table_name}"""
        print(sql)
        rs = db_connection.execute( text(sql))
        for row in rs:
            print(row)
    except Exception as e:
        print("Error: " + str(e))
    finally:
        # close the db connection
        db_connection.close()

    

SELECT FIRST_NAME,BIRTHDATE,Gender,HEALTHCARE_COVERAGE FROM MITRE_PATIENT
('Louie190', datetime.datetime(1973, 7, 4, 0, 0), 'M', Decimal('10929.18'))
('Marvel321', datetime.datetime(1965, 10, 15, 0, 0), 'F', Decimal('4353'))
('Luciana251', datetime.datetime(1993, 3, 9, 0, 0), 'F', Decimal('4274.64'))
('Ashton21', datetime.datetime(1967, 6, 14, 0, 0), 'F', Decimal('11811.21'))
('Erwin847', datetime.datetime(1987, 8, 3, 0, 0), 'M', Decimal('11799.2'))
('Mabel261', datetime.datetime(1986, 8, 24, 0, 0), 'F', Decimal('860.22'))
('Victor265', datetime.datetime(2005, 11, 30, 0, 0), 'M', Decimal('4562.12'))
('Wesley533', datetime.datetime(1967, 6, 14, 0, 0), 'M', Decimal('5593.56'))
('Dorothea248', datetime.datetime(2018, 7, 19, 0, 0), 'F', Decimal('1033.28'))
('Emerson869', datetime.datetime(1959, 5, 20, 0, 0), 'M', Decimal('6321.96'))
('Christene303', datetime.datetime(1994, 5, 31, 0, 0), 'F', Decimal('5137.84'))
('Stanford577', datetime.datetime(1969, 2, 22, 0, 0), 'M', Decimal('3480.56'))


TODOs:

    - Concadinate the string sections into one document 

    - translate the template document into a dictionary with table name and the columns we need to select

    - parse the concadinated document with data comeback from the database

    - store the offset and other informations into a dictionary

    - store the files to xxx.txt

